### Imports

In [10]:
import os
import pandas as pd
import numpy as np
from csv import writer

### Loading Data into Text File

In [11]:
folder = "/Users/reeteshsudhakar/Documents/GitHub/utk-image-classifier/utkface-images"

files = os.listdir(folder)
# grabbing the info from each file name and printing it  

# opening the text file
with open("image_data.txt", "w") as textfile, open("image_data.csv", "w") as csvfile:
    file_writer = writer(csvfile)
    file_writer.writerow(["Age", "Gender", "Race", "Date & Time"])
    for file in files:
        output_list = []
        file = file.strip(".jpg.chip.jpg")
        counter = 0

        for item in file.split("_"):
            textfile.write(item + "\n")
            output_list.append(item)
            counter += 1 

            if counter == 4:
                textfile.write("-----------\n")
                file_writer.writerow(output_list)
                continue
    
    outfile.close()
    # TODO - modify this to grab the info and add it to a table 

### Image Labels
The labels of each face image is embedded in the file name, formated like [age]_[gender]_[race]_[date&time].jpg

**age** is an integer from 0 to 116, indicating the age

**gender** is either 0 (male) or 1 (female)

**race** is an integer from 0 to 4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern).

**date&time** is in the format of yyyymmddHHMMSSFFF, showing the date and time an image was collected to UTKFace

### Dataframe Testing

In [16]:
df = pd.read_csv("image_data.csv")

df

,Age,Gender,Race,Date & Time
0,9,1,2,20161219204347420
1,36,0,1,20170117163203851
2,86,1,0,20170120225751953
3,26,1,0,20170116171048641
4,1,1,2,20161219154612988
...,...,...,...,...
23700,35,0,0,20170117155112202
23701,52,0,3,20170119200211340
23702,26,0,1,20170116010114628
23703,28,0,1,20170117015458481
